# Consumer Databrick for Finance Group:
Contributors: yvcvs, akittendorf, dbode777, TKorby

## Module Imports

In [0]:
from time import sleep
import json
import uuid
import datetime
from confluent_kafka import Consumer, KafkaError, KafkaException
import pandas as pd
import numpy as np
from sklearn.impute import SimpleImputer

## Error Handling Functions

In [0]:
def error_cb(err):
    """ The error callback is used for generic client errors. These
        errors are generally to be considered informational as the client will
        automatically try to recover from all errors, and no extra action
        is typically required by the application.
        For this example however, we terminate the application if the client
        is unable to connect to any broker (_ALL_BROKERS_DOWN) and on
        authentication errors (_AUTHENTICATION). """

    print("Client error: {}".format(err))
    if err.code() == KafkaError._ALL_BROKERS_DOWN or \
       err.code() == KafkaError._AUTHENTICATION:
        # Any exception raised from this callback will be re-raised from the
        # triggering flush() or poll() call.
        raise KafkaException(err)
        
def msg_reader(some_list, msg):
    """
    A Kafka message reader that takes in a message as msg and a list to store the messages in. If the message is not None, the contents of the message, 
    as well as a timestamp of when the message was retrieved, are appended to the list parameter as a JSON object.
    If the message contains an error, the error for that message is printed out.
    This function only works inside of a while loop.
    """
    if msg is None:
        return False
    elif msg.error():
        print(f"Consumer error: {msg.error()}")
        return False
    else:
        rows = json.loads(f"{msg.value().decode('utf-8')}")
        rows['timestamp'] = msg.timestamp()[1]
        some_list.append(rows)
        return True

# Consumer - Setup

In [0]:
# ask tom if all of these confluent variables need to be hidden -Amy 2/5/22
confluentClusterName = "stage3talent"
confluentBootstrapServers = "pkc-ldvmy.centralus.azure.confluent.cloud:9092"
confluentTopicName = "sfb-blob"
schemaRegistryUrl = "https://psrc-gq7pv.westus2.azure.confluent.cloud"

confluentApiKey = dbutils.secrets.get(scope = 'sfb_blob', key = 'mario') # Insert Confluent API Key Here
confluentSecret = dbutils.secrets.get(scope = 'sfb_blob', key = 'fawful') # Insert Confluent Secret Here
confluentRegistryApiKey = dbutils.secrets.get(scope = 'sfb_blob', key = 'luigi') # Insert Confluent Registry Key Here
confluentRegistrySecret = dbutils.secrets.get(scope = 'sfb_blob', key = 'peach') # Insert Confluent Registry Secret Here

In [0]:
#Kakfa Class Setup.
c = Consumer({
    'bootstrap.servers': confluentBootstrapServers,
    'sasl.mechanism': 'PLAIN',
    'security.protocol': 'SASL_SSL',
    'sasl.username': confluentApiKey,
    'sasl.password': confluentSecret,
    'group.id': str(uuid.uuid1()),  # this will create a new consumer group on each invocation.
    'auto.offset.reset': 'earliest',
    'enable.auto.commit': True,
    'error_cb': error_cb,
})

c.subscribe(['sfb-blob'])

In [0]:
clientSecret = dbutils.secrets.get(scope = 'sfb_blob', key = 'bowser')
clientid = dbutils.secrets.get(scope = 'sfb_blob', key = 'yoshi') 

storageAccount = "gen10datafund2111"
storageContainer = "superfinancebros"
mount_point = "/mnt/superfinancebros/capstone"

# Configuration for mount point
configs = {"fs.azure.account.auth.type": "OAuth",
       "fs.azure.account.oauth.provider.type": "org.apache.hadoop.fs.azurebfs.oauth2.ClientCredsTokenProvider",
       "fs.azure.account.oauth2.client.id": clientid,
       "fs.azure.account.oauth2.client.secret": clientSecret,
       "fs.azure.account.oauth2.client.endpoint": "https://login.microsoftonline.com/d46b54b2-a652-420b-aa5a-2ef7f8fc706e/oauth2/token",
       "fs.azure.createRemoteFileSystemDuringInitialization": "true"}

try: 
    dbutils.fs.unmount(mount_point)
except:
    pass

dbutils.fs.mount(
source = "abfss://"+storageContainer+"@"+storageAccount+".dfs.core.windows.net/",
mount_point = mount_point,
extra_configs = configs)

/mnt/superfinancebros/capstone has been unmounted.
Out[48]: True

In [0]:
ETF_stocks = ['VTI','VGT','VIS','VHT','VFH','VCR']

# {"name": "Real Gross Domestic Product", "interval": "annual", "unit": "billions of dollars", "data": []}
messages = []
work = True
while work:
    try:
        msg = c.poll(timeout=1.0)
        work = msg_reader(messages, msg)
    except Exception as e: # Outer except statement
        print(e)

In [0]:
month_stock_info = []
economic_features = []

for message in messages:
    if 'Meta Data' in message.keys():
        month_stock_info.append(message)
    else:
        economic_features.append(message)
try:       
    month_stock_info =  month_stock_info[-6:]
    economic_features = economic_features[-7:]
except Exception as e:
    print(e)

## Cleaning the Kafka Messages

This section will create all desired data frames that will be used for our ML model and for our visualizations. 

Data frames created include:
  - One that combines ETF stocks percentage change in closing prices with the economic features, condensed to a monthly time scale, and null values imputed with the averages for each feature.
  - One that combines the closing prices of all ETF stocks and the US revenue data, condensed to an annual time scale. 
  
This should contain all the information needed for our visualizations.

### Annual Stock Price and US Revenue Cleaning

In [0]:
# Data Cleaning Steps for ETL
import numpy as np
mean_fit = SimpleImputer(missing_values=np.nan,strategy='mean')

try:
    stock_dict = {}
    for stock in range(0,len(month_stock_info)):
        new_dict = {}
        for date in list(month_stock_info[stock]['Monthly Time Series'].keys()): #Looks at every date for a given stock
            new_dict[date] = month_stock_info[stock]['Monthly Time Series'][date]['4. close'] # Saves the closing price for that date in a dictionary that has all closing prices a stock

        # Saves the closing price dictionary to a key named after the stock (i.e. {'VTI': {'2022-01-01':'55', ...}, ...})
        stock_dict[ETF_stocks[stock]] = new_dict

    # Data Cleaning Steps for ETL
    new_df = pd.DataFrame(stock_dict)

    for x in list(new_df.columns):
        new_df[x] = pd.to_numeric(new_df[x],downcast='float')

# Impute null values in the event that the dates of recorded data aren't the same for each ETF stock
    for column in list(new_df.columns):
        new_df[column] = mean_fit.fit_transform(new_df[column].values.reshape(-1,1))

    # Sorting the monthly data to be between the years 2006 and 2020
    new_df['Year'] = list(map(lambda date: int(date[0:4]), list(new_df.index))) # Create year column to filter out years prior to 2006
    new_df = new_df[(2005<new_df['Year']) & (new_df['Year']<=2020)]
    new_df['Year'] = pd.to_numeric(new_df['Year'],downcast='integer')

    # Grouping by year and merging the US census data with the annual stock prices
    annual_df = new_df.groupby('Year').mean()
    annual_df['Year'] = list(annual_df.index)
    annual_df.rename_axis('',axis=0,inplace=True)

    US_rev = pd.read_csv(f"/dbfs{mount_point}/USrevenue.csv")
    US_rev['Year'] = pd.to_numeric(US_rev['Year'])
    annual_df['US_rev'] = US_rev['United States'].values # Ready to load into SQL

    #Save data to blob container as a backup file
    annual_df.to_json(f"/dbfs{mount_point}/Cleaned_Dataframes/annual_data.json")
    
except Exception as e:
    print(e)

### Monthly Stock Price Cleaning

Combines economic feature data with pct_change data

In [0]:
# Data Cleaning Steps for ETL
try:
    df = pd.DataFrame(month_stock_info[0]['Monthly Time Series'])
    df = df.T
    for x in range(0, len(list(df.columns))):
        df.rename({df.columns[x] : df.columns[x][3:]},axis='columns',inplace=True)

    for x in list(df.columns):
        df[x] = pd.to_numeric(df[x],downcast='float')
        
    copy_df = df.copy()
    # set index dates
    # create series of VTI dates
    vtiDates = pd.Series(copy_df.index)
    vtiDates = vtiDates.apply(lambda date: datetime.date(int(date[0:4]),int(date[5:7]),1))

    # set index
    copy_df.set_index(vtiDates, inplace=True)

    for x in range(0,len(economic_features)):
        dates = map(lambda date: date['date'][:7], economic_features[x]['data'])
        values = map(lambda value: value['value'], economic_features[x]['data'])
        
        indicator = pd.DataFrame(values,index=dates,columns=[economic_features[x]['name']])
        indicator.replace('.',np.nan,inplace=True) #Replacing . entries with null values for imputation later

        # format the date index
        # create series of inidcator dates
        indicatorDates = pd.Series(indicator.index)
        indicatorDates = indicatorDates.apply(lambda date: datetime.date(int(date[0:4]),int(date[5:7]),1))

        # set index to be dates
        indicator.set_index(indicatorDates, inplace=True)

        indicator[economic_features[x]['name']] = pd.to_numeric(indicator[economic_features[x]['name']])
        copy_df = copy_df.join(indicator, how='outer')
        
    copy_df['Year'] = list(map(lambda date: int(str(date)[0:4]),copy_df.index)) # Create year column to filter out years prior to 2006
    modern_df = copy_df[copy_df['Year']>2005]
    
    # Impute null values with the averages for each column
    for column in list(modern_df.columns):
        modern_df[column] = mean_fit.fit_transform(modern_df[column].values.reshape(-1,1)) # Ready to load into a SQL table
    
# Merges the percentage change of all ETF stocks to modern_df
    monthly_df = pd.DataFrame(stock_dict)
    indicatorDates = pd.Series(monthly_df.index)
    indicatorDates = indicatorDates.apply(lambda date: datetime.date(int(date[0:4]),int(date[5:7]),1))

    # set index to be dates
    monthly_df.set_index(indicatorDates, inplace=True)

    for x in list(monthly_df.columns):
        monthly_df[x] = pd.to_numeric(monthly_df[x],downcast='float')

    # Add year column to filter by
    monthly_df['Year'] = list(map(lambda date: int(str(date)[0:4]), monthly_df.index)) # Create year column to filter out years prior to 2004
    monthly_df = monthly_df[monthly_df['Year']>2005]

    monthly_df = modern_df.join(monthly_df.drop(['Year'],axis=1),how='outer')
    monthly_df.drop(["open",'high','low','close','volume','Year'],axis=1,inplace=True) 
    monthly_df['date'] = list(monthly_df.index)
    monthly_df.reset_index(inplace=True)
    monthly_df.drop('index',axis=1,inplace=True)
    
    # Save to blob container in JSON format
    monthly_df.to_json(f"/dbfs{mount_point}/Cleaned_Dataframes/monthly_data.json")
except Exception as e:
    print(e)

<command-522247351784039>:43: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
 modern_df[column] = mean_fit.fit_transform(modern_df[column].values.reshape(-1,1)) # Ready to load into a SQL table
<command-522247351784039>:43: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
 modern_df[column] = mean_fit.fit_transform(modern_df[column].values.reshape(-1,1)) # Ready to load into a SQL table
<command-522247351784039>:43: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
 modern_df[column] = mean_fit.fit_transform(modern_df[column].values.reshape(-1,1)) # Ready to load into a SQL table
<command-522247351784039>:43: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
 modern_df[column] = mean_fit.fit_transform(modern_df[column].values.reshape(-1,1)) # Ready to load into a SQL table
<command-522247351784039>:43: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
 modern_df[column] = mean_fit.fit_transform(modern_df[column].values.reshape(-1,1)) # Ready to load into a SQL table
<command-522247351784039>:43: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
 modern_df[column] = mean_fit.fit_transform(modern_df[column].values.reshape(-1,1)) # Ready to load into a SQL table
<command-522247351784039>:43: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
 modern_df[column] = mean_fit.fit_transform(modern_df[column].values.reshape(-1,1)) # Ready to load into a SQL table
<command-522247351784039>:43: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
 modern_df[column] = mean_fit.fit_transform(modern_df[column].values.reshape(-1,1)) # Ready to load into a SQL table
<command-522247351784039>:43: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
 modern_df[column] = mean_fit.fit_transform(modern_df[column].values.reshape(-1,1)) # Ready to load into a SQL table
<command-522247351784039>:43: SettingWithCopyWarning: 
A value is trying to be set on

## Loading the Data Frames into our SQL Database:

In [0]:
# Loading for ETL document

server="gen10-data-fundamentals-21-11-sql-server.database.windows.net"
database="Super-Financial-Bros"
username=dbutils.secrets.get(scope = 'sfb_blob', key = 'birdo') # Insert Username Here
password=dbutils.secrets.get(scope = 'sfb_blob', key = 'donkeykong') # Insert Password Here

spark_annual_df = spark.createDataFrame(annual_df)
spark_monthly_df = spark.createDataFrame(monthly_df)

spark_annual_df.write.format('jdbc').option("url", f"jdbc:sqlserver://{server}:1433;databaseName={database};") \
    .mode("overwrite") \
    .option("dbtable", "annualData") \
    .option("user", username) \
    .option("password", password) \
    .option("driver", "com.microsoft.sqlserver.jdbc.SQLServerDriver") \
    .save()

spark_monthly_df.write.format('jdbc').option("url", f"jdbc:sqlserver://{server}:1433;databaseName={database};") \
    .mode("overwrite") \
    .option("dbtable", "monthlyData") \
    .option("user", username) \
    .option("password", password) \
    .option("driver", "com.microsoft.sqlserver.jdbc.SQLServerDriver") \
    .save()